In [3]:
import sys
sys.path.append('/home/jra21/work/source/repos/opep')

In [4]:
from openep import io as openep_io
from openep import case as openep_case
from openep import mesh_routines as openep_mesh
from openep import case_routines as case_routines

import numpy as np
from scipy.interpolate import LinearNDInterpolator as linterp
from scipy.interpolate import NearestNDInterpolator as nearest
from sklearn.neighbors import NearestNeighbors
from matplotlib.cm import jet, rainbow, jet_r, seismic
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'sklearn'

In [2]:
# # Usecase 039 - Creating a voltage map from electroanatomic mapping data

filename = '/home/jra21/work/source/repos/opep/examples/data/new_dataset_1.mat'
distanceThresh = 10


ep_case = openep_io.load_case(filename)
ep_case_mesh = ep_case.create_mesh()
pts = ep_case.nodes

print(ep_case)

NameError: name 'openep_io' is not defined

In [ ]:
# # Load EGMx and EGM voltage values
coords = ep_case.electric['egmX'].T
data = ep_case.['egm'].T


iVp = case_routines.getMappingPointsWithinWoI(ep_case)
# macthing the shape of ivp with data
iVp_data = np.repeat(iVp, repeats=data.shape[1], axis=1)
# macthing the shape of ivp with coords
iVp_coords = np.repeat(iVp, repeats=coords.shape[1],axis=1)

data[~iVp_data] = np.nan
coords[~iVp_coords] = np.nan


# For each mapping point, n, find the voltage amplitude
max_volt = np.amax(a=data,axis=1).reshape(len(data),1)
min_volt = np.amin(a=data,axis=1).reshape(len(data),1)


amplitude_volt = np.subtract(max_volt,min_volt)


In [ ]:
# Remove any data with Nans
for indx in range(amplitude_volt.shape[1]):
    tempData = amplitude_volt[:,indx]
    tempCoords = coords
    iNaN = np.isnan(tempData)
    tempData=tempData[~iNaN]
    tempCoords=tempCoords[~iNaN]



    interp = case_routines.OpenEPDataInterpolator(method='rbf',distanceThreshold=distanceThresh,rbfConstant=1)
    VertexVoltageData = interp.interpolate(x0=tempCoords,d0=tempData,x1=pts)
    
    
#     Rbf - Scipy 
    d1_py = case_routines.rbf_scipy(x0=tempCoords,d0=tempData,x1=pts,rbfConstant=1)
    #print(c)

ep_case.fields['d1'] = VertexVoltageData


In [ ]:
    
# DRAW Map
openep_mesh.compute_field(mesh=ep_case_mesh,fieldname='d1',minval=0,maxval=1,color_map=jet_r)
ep_case_mesh.show()

### Matlab - d1

In [ ]:
sys.path.append('/home/jra21/work/source/repos/openep-core')
filename = '/home/jra21/work/source/repos/openep-core/py_d1.mat'
mat_d1_file = openep_io.load_mat(filename)
mat_d1 = mat_d1_file['py_d1'].flatten()

mat_d1_NotNan = mat_d1[~np.isnan(mat_d1)]
print(mat_d1_NotNan.shape)

for item in mat_d1_NotNan:
    print(item)

In [ ]:
x = range(len(mat_d1_NotNan))
y = mat_d1_NotNan
plt.plot(x,y,'ro')

### Py_code - d1

In [ ]:
VertexVoltageData_NotNan = VertexVoltageData[~np.isnan(VertexVoltageData)]
print(VertexVoltageData_NotNan.shape)
for item in VertexVoltageData_NotNan:
    print(item)

In [ ]:
x = range(len(VertexVoltageData_NotNan))
y = VertexVoltageData_NotNan
plt.plot(x,y,'bo')

### SciPy - rbf

In [ ]:
d1_py

print(d1_py.shape)

for item in d1_py:
    print(item)

In [ ]:
x = range(len(d1_py))
print(x)
y = d1_py
plt.plot(x,y,'go')


### difference between Mat d1 and Py d1

In [ ]:
diff = np.subtract(VertexVoltageData_NotNan,mat_d1_NotNan)
print(np.absolute(diff).max())

### Trimesh Plot with FreeBoundary/Outline

In [ ]:
import trimesh


# print(ep_case)
nodes = ep_case.nodes
# print(nodes)
indices = ep_case.indices
# print(indices)

mesh = trimesh.Trimesh(vertices=nodes,faces=indices)

freeBoundary = mesh.outline().to_dict()
inds = np.array(freeBoundary['entities'][0]['points'])


print(indices[inds])


# freeBoundary = ep_case_mesh.outline()
# # vertex_index = ep_case_mesh.outline().vertex_nodes()

# print(dir(freeBoundary))
# print(freeBoundary.vertex_nodes)


scene = trimesh.Scene([ep_case_mesh])
scene.show()